# 1. 붓꽃 판별

### 1.1 패키지 로딩

In [37]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score, f1_score,classification_report,roc_auc_score

### 1.2 데이터 로드 및 확인

In [21]:
X, y = load_iris(return_X_y=True)
print(X.shape, y.shape)
unique, counts = np.unique(y,return_counts=True)
print(dict(zip(unique,counts)))

(150, 4) (150,)
{0: 50, 1: 50, 2: 50}


In [22]:
df = pd.DataFrame(X, columns=load_iris().feature_names)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


### 1.3 학습 데이터와 평가 데이터 분할

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,random_state=10, stratify=y)

In [12]:
print('y의 클래스별 데이터 수: ',np.bincount(y))
print('y_train의 클래스별 데이터 수: ',np.bincount(y_train))
print('y_test 의 클래스별 데이터 수: ',np.bincount(y_test))

y의 클래스별 데이터 수:  [50 50 50]
y_train의 클래스별 데이터 수:  [40 40 40]
y_test 의 클래스별 데이터 수:  [10 10 10]


### 모델 생성
- 다항분류를 위한 옵션
  - solver: 최적화 문제에 사용할 알고리즘 (default: lbfgs)
    >- 'newton-cg', 'sag', 'saga', 'lbfgs' 만 다항 손실을 처리한다.
    >- 'liblinear' 는 one-versus-rest 방식으로 제한된다.
  - multi_class: 다중클래스분류 문제의 상황에서 어떤 접근 방식을 취할지 결정
    >- 'ovr': 이진분류기인 sigmoid 함수를 이용하여 결과 예측
    >- 'multinomial': 각 클래스에 대한 softmax 함수를 이용하여 다중분류를 수행

In [24]:
model = LogisticRegression(random_state=0,solver='lbfgs',multi_class='ovr',max_iter=2000)

model.fit(X_train,y_train)


LogisticRegression(max_iter=2000, multi_class='ovr', random_state=0)

### 1.5 모델 예측

In [36]:
y_hat = model.predict(X_test)

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [26]:
# 예측 확률을 구함
proba = model.predict_proba(X_test)
print(np.round(proba[:5],3))

[[0.016 0.735 0.248]
 [0.905 0.095 0.   ]
 [0.02  0.574 0.406]
 [0.003 0.439 0.558]
 [0.238 0.754 0.008]]


In [29]:
df_proba = pd.DataFrame(proba[:5], columns=['class-0','class-1','class-2'])
df_proba['result']= np.argmax(proba[:5],axis=1)
print(y_hat[:5])
df_proba

[1 0 1 2 1]


,class-0,class-1,class-2,result
0,0.016071,0.735477,0.248452,1
1,0.904712,0.095275,0.000013,0
2,0.020361,0.573742,0.405898,1
3,0.002988,0.439262,0.557750,2
4,0.237762,0.754042,0.008196,1


### 1.6 모델 평가

In [34]:
print(f'Accuracy:{accuracy_score(y_test,y_hat):.3f}')
# ovo: One-Versus-One 방식으로 모든 고유한 쌍별 클래스 조합을 비교한다.
auc = roc_auc_score(y_test, proba, multi_class='ovo')
print(f'AUC:{auc:.3f}')
con_mat = confusion_matrix(y_test, y_hat)
print(con_mat)
report = classification_report(y_test,y_hat)
print(report)

Accuracy:1.000
AUC:1.000
[[10  0  0]
 [ 0 10  0]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00        10
           2       1.00      1.00      1.00        10

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



In [35]:
model = LogisticRegression(random_state=0,solver='lbfgs',multi_class='multinomial',max_iter=2000)


### 1.7 하이퍼 파라메터 튜닝